In [1]:
import numpy as np
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris()['data'], load_iris()['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1234)

X_train, X_test = X_train.T, X_test.T
y_train = y_train.reshape(1, len(y_train))
y_test = y_test.reshape(1, len(y_test))

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4, 120) (1, 120)
(4, 30) (1, 30)


In [2]:
def OneHotEncode(y):
    '''
    check its shape if its in (1,m)
    '''
    shape=(y.shape[1], y.max()+1)
    OHE = np.zeros(shape)
    rows = np.arange(y.shape[1])
    OHE[rows, y]=1
    
    return OHE

### Multi Class Logistic Regression

In [3]:
class MultClassLogisticRegression:
    def __init__(self, X, y):
        self.X = X
        self.y = OneHotEncode(y).T
        '''
        Though in the final code, just check, whether it is
        having One-Hot-Encoding or not
        '''
        self.W = np.random.randn(self.y.shape[0], self.X.shape[0])
        self.b = np.zeros(shape=(1, self.X.shape[1]))
    
    def _feed_forward(self):
        Z = np.dot(self.W, self.X) + self.b
        return Z
    
    def _softmax(self, X):
        if X.shape[0] < X.shape[1]:
            X = X.T
        X_exp = np.exp(X)
        for i in range(0, X_exp.shape[0]-1):
            X_exp[i:i+1] = X_exp[i:i+1]/X_exp[i:i+1].sum()
        return X_exp.T
    
    def accuracy(self, predictions, ground_truth):
        ground_truth = np.argmax(ground_truth, axis=0)
        predictions = np.argmax(predictions, axis=0)
        number_example = predictions.shape[0]
        accuracy = (np.sum(np.round(predictions) == ground_truth)) / number_example
        return accuracy
    
    def _categorical_cross_entropy(self, Y_pred):
        m = Y_pred.shape[1]
        return -1/m * np.sum(self.y * np.log(Y_pred))
    
    def _compute_grads(self, Y_pred):
        W_grad = np.dot((Y_pred - self.y), self.X.T)
        b_grad = np.sum((Y_pred - self.y), axis=0, keepdims = True)
        
        return (W_grad, b_grad)
    
    def train(self, learning_rate=0.01, epochs=100):
        for epoch in range(epochs):
            Z_pred = self._feed_forward()
            Y_pred = self._softmax(Z_pred)
            cost = self._categorical_cross_entropy(Y_pred)
            W_grad, b_grad = self._compute_grads(Y_pred)

            self.W -= learning_rate * W_grad
            self.b -= learning_rate * b_grad

            if epoch % 50 == 0:
                print(f"After epoch {epoch} cost: {cost}, acc: {self.accuracy(Y_pred, self.y)}")

In [4]:
regressor = MultClassLogisticRegression(X_train, y_train)

In [5]:
regressor.train(learning_rate=0.003, epochs=400)

After epoch 0 cost: 1.5720537707331257, acc: 0.3
After epoch 50 cost: 0.2324935208877936, acc: 0.9416666666666667
After epoch 100 cost: 0.4156266124466423, acc: 0.8166666666666667
After epoch 150 cost: 0.6374676604180873, acc: 0.7666666666666667
After epoch 200 cost: 1.879002929798373, acc: 0.6916666666666667
After epoch 250 cost: 0.6862534901364491, acc: 0.8083333333333333
After epoch 300 cost: 0.1287111518183958, acc: 0.9583333333333334
After epoch 350 cost: 0.12329810412482332, acc: 0.9583333333333334


In [6]:
# test with images in colab